# (7) Getting Coursera Students Interactions

* **author** = Diego Sapunar-Opazo
* **copyright** = Copyright 2019, Thesis M.Sc. Diego Sapunar - Pontificia Universidad Católica de Chile
* **credits** = Diego Sapunar-Opazo, Ronald Perez, Mar Perez-Sanagustin, Jorge Maldonado-Mahauad
* **maintainer** = Diego Sapunar-Opazo
* **email** = dasapunar@uc.cl
* **status** = Dev

This scripts gets the Coursera Report, Coursera Gradebook and creates:

Coursera_learning_path.csv

(1) **num_alumno**, which corresponds to the internal face-to-face students' id and 

(2) **week**, which corresponds to the week

(3) **session**, which corresponds to a study session

(4) **timestamp**, which corresponds to the timestamp when the interaction was recorded

(5) **interaction**, which corresponds to the type of interaction

(6) **timespent**, which corresponds how long was that interaction in seconds


## Part 0: Import Packages

In [1]:
# data analysis and wrangling
import pandas as pd
import numpy as np

# datetime
from datetime import datetime

SESSION_LENGTH = 45 * 60 # in seconds

WEEKS_METADATA = {
                1: (datetime.strptime('2018-08-20 00:00:01.00', '%Y-%m-%d %H:%M:%S.%f'), 
                    datetime.strptime('2018-08-26 23:59:59.00', '%Y-%m-%d %H:%M:%S.%f')),
                2: (datetime.strptime('2018-08-27 00:00:01.00', '%Y-%m-%d %H:%M:%S.%f'),
                    datetime.strptime('2018-09-02 23:59:59.00', '%Y-%m-%d %H:%M:%S.%f')),
                3: (datetime.strptime('2018-09-03 00:00:01.00', '%Y-%m-%d %H:%M:%S.%f'),
                    datetime.strptime('2018-09-09 23:59:59.00', '%Y-%m-%d %H:%M:%S.%f')),
                4: (datetime.strptime('2018-09-10 00:00:01.00', '%Y-%m-%d %H:%M:%S.%f'),
                    datetime.strptime('2018-09-16 23:59:59.00', '%Y-%m-%d %H:%M:%S.%f')),
                5: (datetime.strptime('2018-09-24 00:00:01.00', '%Y-%m-%d %H:%M:%S.%f'),
                    datetime.strptime('2018-09-30 23:59:59.00', '%Y-%m-%d %H:%M:%S.%f')),
                6: (datetime.strptime('2018-10-01 00:00:01.00', '%Y-%m-%d %H:%M:%S.%f'),
                    datetime.strptime('2018-10-07 23:59:59.00', '%Y-%m-%d %H:%M:%S.%f')),
                7: (datetime.strptime('2018-10-08 00:00:01.00', '%Y-%m-%d %H:%M:%S.%f'),
                    datetime.strptime('2018-10-14 23:59:59.00', '%Y-%m-%d %H:%M:%S.%f')),
                8: (datetime.strptime('2018-10-22 00:00:01.00', '%Y-%m-%d %H:%M:%S.%f'),
                    datetime.strptime('2018-10-28 23:59:59.00', '%Y-%m-%d %H:%M:%S.%f')),
                9: (datetime.strptime('2018-10-29 00:00:01.00', '%Y-%m-%d %H:%M:%S.%f'),
                    datetime.strptime('2018-11-04 23:59:59.00', '%Y-%m-%d %H:%M:%S.%f')),
                101: (datetime.strptime('2018-11-12 00:00:01.00', '%Y-%m-%d %H:%M:%S.%f'),
                       datetime.strptime('2018-11-18 23:59:59.00', '%Y-%m-%d %H:%M:%S.%f')),
                102: (datetime.strptime('2018-11-05 00:00:01.00', '%Y-%m-%d %H:%M:%S.%f'),
                       datetime.strptime('2018-11-11 23:59:59.00', '%Y-%m-%d %H:%M:%S.%f')),
                11: (datetime.strptime('2018-11-19 00:00:01.00', '%Y-%m-%d %H:%M:%S.%f'),
                       datetime.strptime('2018-11-25 23:59:59.00', '%Y-%m-%d %H:%M:%S.%f')),
                }    

## Part 1: Getting the Data

In [2]:
def read_data(path):
    '''
    Read a .csv file and convert it in a Pandas DataFrame.
    
    Input:
    path - String: path where the .csv is located.
    
    Output:
    Pandas DataFrame: .csv in the Pandas DataFrame format.
    '''
    
    return pd.read_csv(path)

## Part 2: Data Preprocessing & Wrangling

In [3]:
def preprocc_data(df, slices=False, columns_to_rename=False, datetime=False):
    '''
    From a dataframe on the fly, (1) get the necessary columns; (2) rename columns; and (3) clean data.
    
    Input: 
    df - Pandas DataFrame: dataframe to be cleaned.
    columns_to_rename - Dict: Columns to rename, Key: original name, Value: new name.
    datetime - List of Strings: List of the names of the columns to be datetime.
    
    Output:
    df - Pandas DataFrame: the dataframe already cleaned.
    '''
    
    df_cleaned = df.copy()
    
    # slicing the columns, getting only the one that I need (num_alumno and seccion)
    if slices:
        df_cleaned = df_cleaned.iloc[:,slices]
    
    del df  # clean memory
    
    # rename columns
    if columns_to_rename:
        df_cleaned.rename(_columns_to_rename, 
                          inplace=True, 
                          axis=1)
    
    if datetime:
        for cat in datetime:
            
            df_cleaned[cat] = pd.to_datetime(df_cleaned[cat], 
                                             format='%Y-%m-%d %H:%M:%S.%f')
    
    return df_cleaned

In [4]:
def merging(df1, df2, variable1, variable2):
    '''
    Merge df1 and df2 over the variable.
    
    Input:
    df1 - Pandas DataFrame
    df2 - Pandas DataFrame
    variable - String: name of the column to use as pivot.
    
    Output:
    Pandas DataFrame
    '''
    
    # getting same types
    df1[variable1] = df1[variable1].astype('str')
    df2[variable2] = df2[variable2].astype('str')
    
    return pd.merge(left=df1, right=df2, left_on=variable1, right_on=variable2)

In [5]:
def get_week(timestamp, sec, WEEKS_METADATA):
    '''
    Get the week that corresponds to the timestamp.
    
    Input:
    timestamp - datetime: corresponds to the timestamp of the interaction
    sec - int: corresponds to the section of the user
    
    Output:
    week - int
    '''
    
    for week in range(1,12):
        if week == 10:
            if sec == 1: week = 101
            else: week = 102
        
        if (timestamp > WEEKS_METADATA[week][0]) and (timestamp <= WEEKS_METADATA[week][1]):
            if week in [101, 102]: week = 10
            return week

In [6]:
def get_obtained(df_grades, lms_id, item_id):
    '''
    Get the timespant when the user get his better grade and if he pass or not that item.
    
    Input:
    df_grades - pandas DataFrame: the dataframe with de grades.
    lms_id - String: lms_id to be consulted.
    item_id - String: item_id of the item to be evaluated.
    
    Output:
    Tuple with the timestamp when the grade was obtained and if the student pass or don't the item
    '''
    
    aux = df_grades[(df_grades['lms_id']==lms_id) & (df_grades['item_id']==item_id)]
    if len(aux):
        if item_id: return (aux.iloc[0][2], aux.iloc[0][3])
        else: return (aux.iloc[0][2], aux.iloc[0][3])
    else:
        return None

In [7]:
def get_interaction(item_id, item_state, videos, 
                    readings, quiz, items_completed_dic, 
                    item_ts, df_grades, lms_id):
    '''
    Return the type of the interaction of the item.
    
    Input:
    item_id - String: item_id of the item to be evaluated.
    item_state - int: state of the item to be evaluated.
    videos - Series: list of videos.
    readings - Series: list of readings.
    quiz - Series: list of quizes.
    items_completed - list of strings: list of the item_ids completed
    item_ts - DateTime: the datetime of the item
    
    Output:
    String with the type of interaction
    '''
    
    items_completed = items_completed_dic[lms_id]
    if item_id in videos.values:
        if item_id in items_completed:
            return 'Video-Lecture review'
        elif item_state == 1:
            return 'Video-Lecture begin'
        else:  # completed
            items_completed.append(item_id)
            items_completed_dic[lms_id] = items_completed
            return 'Video-Lecture completed'
        
    elif item_id in readings.values:
        if item_id in items_completed:
            return 'Reading review'
        elif item_state == 1:
            return 'Reading begin'
        else:  # completed
            items_completed.append(item_id)
            items_completed_dic[lms_id] = items_completed
            return 'Reading completed'
        
    else:  # quiz
        if item_id in items_completed:
            return 'Assessment review'
        else:
            # get grade_obtained_ts
            aux = get_obtained(df_grades, lms_id, item_id)
            if aux:
                grade_obtained_ts, pass_flag = aux
                if item_ts < grade_obtained_ts:
                    return 'Assessment try'
                elif pass_flag:  # pass
                    items_completed.append(item_id)
                    items_completed_dic[lms_id] = items_completed
                    return 'Assessment pass'
                else: 
                    return 'Assessment try'
            else:
                return None

In [8]:
def add_sess(timespent, session_dic, num_alumno, week, next_week, df):
    '''
    Fix the timespent and add sess
    
    Input:
    timespent - float: the original timespent in the interaction
    session_dic - dic: a dic with key the num_alumno and as a key other dic with key week and value the number of sessions in that week
    num_alumno - string
    week - int
    next_week - int: if the next interaction is in another week (works as a boolean)
    df - pandas Dataframe: with the information of the interactions
    
    Output:
    a dic with the timespent fixed and the session of that interaction
    '''
    
    if timespent < 0:
        
        return {'timespent_real': 0, 
                'session':session_dic[num_alumno][week]}
    elif timespent > SESSION_LENGTH:
        if next_week:
            pass
        else:
            session_dic[num_alumno][week] += 1
        return {'timespent_real': get_mean_week(num_alumno, week, df), 
                'session':session_dic[num_alumno][week]}
    
    else:
        return {'timespent_real': timespent, 
                'session':session_dic[num_alumno][week]}

In [9]:
def get_mean_week(num_alumno, week, df):
    '''
    Get the mean timespent in the week
    
    Input:
    num_alumno - String: student's id
    week - int: week
    df - pandas Dataframe: data to query
    
    Output
    float: mean.
    '''
    
    return (df[(df['timespent'] > 0) & (df['timespent'] < SESSION_LENGTH) & (df['num_alumno'] == num_alumno) & (df['week'] == week)])['timespent'].mean()

## Part 3: Export Data

In [10]:
def export_data(df, path):
    '''
    Export df in .csv fole to the path.
    
    Input:
    df - Pandas DataFrame: dataframe to be exported.
    path - String: path where the .csv will be exported.
    '''
    
    df.to_csv(path, index=False)

## Part 4: Main

### Part 4.1: Getting the data

In [11]:
_progress_path = '../../data/raw_data/coursera/coursera_logs/course_progress.csv'
df_progress = read_data(_progress_path)

_columns_to_rename = {
    'course_item_id': 'item_id',
    'ucchile_user_id': 'lms_id',
    'course_item_grade_verified': 'item_grade',
    'course_progress_state_type_id': 'item_state',
    'course_progress_ts': 'item_timestamp'
}

df_progress = preprocc_data(df_progress, 
                            slices=[i for i in range(1, len(df_progress.columns))], 
                            columns_to_rename=_columns_to_rename, 
                            datetime=['item_timestamp'])
                           

_students_lms_id_path = '../../data/clean_data/students_lms_id.csv'
df_students_lms_id = read_data(_students_lms_id_path)
lms_list = df_students_lms_id['lms_id']

_items_path = '../../data/clean_data/coursera_items.csv'
df_items = read_data(_items_path)
mask_videos = df_items['item_type_id'] == 1
videos = df_items[mask_videos]['item_id']
mask_readings = df_items['item_type_id'] == 3
readings = df_items[mask_readings]['item_id']
mask_quiz = (df_items['item_type_id'] == 6) | (df_items['item_type_id'] == 106)
quiz = df_items[mask_quiz]['item_id']
del df_items

_grades_path = '../../data/raw_data/coursera/coursera_logs/course_item_grades.csv'
df_grades = read_data(_grades_path)

_columns_to_rename = {
    'course_item_id': 'item_id',
    'ucchile_user_id': 'lms_id',
    'course_item_passing_state_id': 'item_state',
    'course_item_grade_ts': 'item_timestamp'
}

df_grades = preprocc_data(df_grades, 
                          slices=[i for i in range(1,5)],
                          columns_to_rename=_columns_to_rename,
                          datetime=['item_timestamp'])

_students_sec = '../../data/clean_data/students_sec.csv'
df_students_sec = read_data(_students_sec)

items_completed_dic = {str(i):[] for i in lms_list}

### Part 4.2: Filtering the data

In [12]:
# Keeping in progress our users
mask_filtering_lms_id = df_progress['lms_id'].isin(lms_list)
df_progress = df_progress.loc[mask_filtering_lms_id].copy()

# Keeping in progress only the elements that I'm working on!
mask_filtering_element = (df_progress['item_id'].isin(videos)) | (df_progress['item_id'].isin(readings)) | (df_progress['item_id'].isin(quiz))
df_progress = df_progress.loc[mask_filtering_element].copy()

### Part 4.3: Preparing the data

In [13]:
# Adding num_alumno
df_progress = merging(df_progress, df_students_lms_id, 'lms_id', 'lms_id')

# Adding sec
df_progress = merging(df_progress, df_students_sec, 'num_alumno', 'num_alumno')

# sorting values
df_progress.sort_values(['lms_id', 'item_timestamp'], inplace=True)

# Adding timespent in progress
df_progress['timespent'] = df_progress['item_timestamp'].shift(-1) - df_progress['item_timestamp']
df_progress['timespent'] = df_progress.apply(lambda x: x['timespent'].total_seconds(),axis=1)




### Part 4.4: Feature Extraction

In [14]:
df_students_interactions = pd.DataFrame()

df_students_interactions['num_alumno'] = (df_progress['num_alumno']).copy()

df_students_interactions['week'] = df_progress.apply(lambda x: get_week(x['item_timestamp'], x['sec'], WEEKS_METADATA), axis=1)

df_students_interactions['timestamp'] = (df_progress['item_timestamp']).copy()

df_students_interactions['interaction'] = df_progress.apply(lambda x: get_interaction(item_id=x['item_id'],
                                                                                      item_state=x['item_state'],
                                                                                      videos=videos,
                                                                                      readings=readings,
                                                                                      quiz=quiz,
                                                                                      items_completed_dic=items_completed_dic,
                                                                                      item_ts=x['item_timestamp'], 
                                                                                      df_grades=df_grades, 
                                                                                      lms_id=x['lms_id']), 
                                                            axis=1)

df_students_interactions['timespent'] = (df_progress['timespent']).copy()


df_students_interactions['next_week'] = df_students_interactions['week'].shift(-1) - df_students_interactions['week']

df_students_interactions.dropna(inplace=True)

session_dic = {str(i):{j:1 for j in range(1,12)} for i in df_students_interactions.num_alumno}
df_students_interactions = df_students_interactions.merge(df_students_interactions.apply(lambda x: pd.Series(add_sess(x['timespent'], session_dic, 
                                                        x['num_alumno'], x['week'], x['next_week'], df_students_interactions)),axis=1), left_index=True, right_index=True)


### Part 4.4: Export Data

In [16]:
_export_path_students_interactions = '../../data/final_data/coursera_students_interactions_nueva.csv'

export_data(df_students_interactions.drop(['timespent', 'next_week'], axis=1).rename({'timespent_real':'timespent'}, axis=1), _export_path_students_interactions)

In [20]:
export_data(aux, _export_path_students_interactions)

In [19]:
aux.dtypes

num_alumno             object
week                  float64
timestamp      datetime64[ns]
interaction            object
timespent             float64
session               float64
dtype: object